In [143]:
#常用库准备：numpy，pandas，matplotlib
from numpy import *
from numpy.linalg import multi_dot
import pandas as pd
import matplotlib.pyplot as plt

In [144]:
#pip install cufflinks

In [145]:
#加载画图库
import cufflinks as cf
cf.set_config_file(
    offline=True,
    dimensions=(1000,6000),
    theme='henanigans')
import plotly.express as px
px.defaults.width = 1000
px.defaults.height = 600

#忽略警告提醒
import warnings 
warnings.filterwarnings('ignore')

In [146]:
#解决中文问题
plt.rcParams['font.sans-serif'] = ['SimHei']

#解决负号问题
plt.rcParams['axes.unicode_minus'] = False

In [147]:
#导入外部数据
#加载数据
file_path = r'C:\Users\xx\Desktop\有效边界和市场组合.csv'
data = pd.DataFrame(pd.read_csv(file_path,index_col=0,parse_dates=True)).dropna()
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\xx\\Desktop\\有效边界和市场组合.csv'

In [ ]:
#将股价按首个交易日进行归一化处理并可视化
(data/data.iloc[0]).plot(figsize =(12,6))
plt.show()

In [ ]:
#按照对数收益率的计算公式得到股票收益率
returns = log(data/data.shift(1)).dropna()

In [ ]:
returns.describe()

In [ ]:
#年化收益率及波动
pd.DataFrame({
    '年化收益':round(returns.mean()*252*100,2),
    '年化波动':round(returns.std()*252*100,2)
}).iplot(kind='bar',title='年化收益率及波动(%)',shared_xaxes=False,subplots= True,layout=dict(height=400,xaxis=dict(title='资产'),yaxis=dict(title='百分比 (%)')))

In [148]:
#定义一个等权重的投资组合
symbols = data.iloc[0].tolist()
numofasset = len(symbols)
weights = numofasset*[1./numofasset]
weights = array(weights)[:,newaxis]
weights

array([[0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667]])

In [149]:
#计算股票的年化收益率
R_mean = array(returns.mean()*252*10)[:,newaxis]
R_mean

array([[3.68221389],
       [0.62244796],
       [1.00889537],
       [1.88494217],
       [0.09857818],
       [0.41606304]])

In [150]:
#计算投资组合的预期收益
R_port = weights.T @ R_mean
print('投资组合的预期收益：',R_port)

投资组合的预期收益： [[1.28552344]]


In [151]:
#计算股票的协方差矩阵并做年化处理
R_cov = returns.cov()
R_cov

,比亚迪,贵州茅台,东方财富,宁德时代,广发证券,中国银河
比亚迪,0.000821,0.000194,0.000358,0.000501,0.000200,0.000179
贵州茅台,0.000194,0.000343,0.000251,0.000216,0.000154,0.000152
东方财富,0.000358,0.000251,0.001039,0.000428,0.000464,0.000465
宁德时代,0.000501,0.000216,0.000428,0.001144,0.000219,0.000218
广发证券,0.000200,0.000154,0.000464,0.000219,0.000452,0.000334
中国银河,0.000179,0.000152,0.000465,0.000218,0.000334,0.000541


In [152]:
#计算投资组合收益波动率
var = multi_dot([weights.T,R_cov,weights])
vol_port = sqrt(var)

print('投资组合收益波动率:',vol_port)

投资组合收益波动率: [[0.01900801]]


In [153]:
#定义一个组合函数
def portfolio_simulation(returns):
    rets = []; vols = [];wts=[]
    #模拟5000个不同权重的组合
    for i in range (5000):
       #产生随机权重
       weights = random.random(numofasset)[:newaxis]
       #设置权重之和为1
       weights /= sum(weights)
       
       #记录组合的收益、波动、权重
       rets.append(weights.T @ array(returns.mean()*252)[:,newaxis])
       vols.append(sqrt(multi_dot([weights.T,returns.cov()*252,weights])))
       wts.append(weights.flatten())
    #记录组合个数据
    portdf = 100*pd.DataFrame({
        'port_rets':array(rets).flatten(),
        'port_vols':array(vols).flatten(),
        'weights':list(array(wts))
         })
    portdf['Sharpe_ratio'] = portdf['port_rets']/portdf['port_vols']
    return round(portdf,2)

In [154]:
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,weights,Sharpe_ratio
0,10.22,32.90,"[6.480684509695646, 6.6744107472615015, 25.66053629078267, 21.223569870664765, 26.340893118847426, 13.619905462747994]",0.31
1,11.06,31.83,"[8.13580266324488, 17.210929705246123, 26.75671125464169, 18.26027658836615, 12.11084423874342, 17.52543554975773]",0.35
2,18.44,33.37,"[29.622732186553886, 7.176798317612006, 14.423694120448676, 25.42699329882348, 4.2411619394749955, 19.108620137086966]",0.55
3,11.61,32.01,"[5.139343167969044, 19.706334119945502, 16.070668690879984, 32.480379122390715, 11.222539969857502, 15.380734928957251]",0.36
4,9.31,27.46,"[13.057442985926876, 24.047160206542184, 7.291580291261962, 7.19954986166804, 34.791421329787276, 13.61284532481367]",0.34


In [155]:
#找夏普比率最高的组合
temp.iloc[temp.Sharpe_ratio.idxmax()]

port_rets                                                                                                                      28.07
port_vols                                                                                                                      37.51
weights         [71.38507100033084, 2.9580939689626105, 8.751255054723686, 1.1949744369189168, 5.071319267266793, 10.63928627179716]
Sharpe_ratio                                                                                                                    0.75
Name: 626, dtype: object

In [156]:
#模拟可视化
fig = px.scatter(
    temp,x='port_vols',y='port_rets',color='Sharpe_ratio',
    labels={'port_vols':'预期波动','PORT_rets':'预期收益','Sharpe_ratio':'夏普比率'},
    title='蒙特卡洛模拟资产组合'
    ).update_traces(mode='markers', marker=dict(symbol='cross'))
#绘制最大夏普比率的组合
fig.add_scatter(
    mode='markers',
    x=[temp.iloc[temp.Sharpe_ratio.idxmax()]['port_vols']],
    y=[temp.iloc[temp.Sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue',size=20,symbol='star'),
    name='市场组合'
    ).update(layout_showlegend=False)
fig.update_xaxes(showspikes=True)
fig.update_xaxes(showspikes=True)
fig.show()